In [2]:
from pathlib import Path

import rasterio
from utils.data_io import get_value_from_raster

from utils.data_io import get_csv

In [3]:
data = get_csv(Path("keypoints_.csv"))
print(data.columns)
# 提取部分数据
ds = data[
    [
        "Latitude (deg)",
        "Longitude (deg)",
        "UTM Easting (m)",
        "UTM Northing (m)",
        "Height (m MSL)",
    ]
]

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'Time (sec)',
       'Delta Time (sec)', 'Segment ID', 'GT Num', 'Beam Num', 'Beam Type',
       'Latitude (deg)', 'Longitude (deg)', 'UTM Easting (m)',
       'UTM Northing (m)', 'UTM Zone', 'UTM Hemisphere', 'Cross-Track (m)',
       'Along-Track (m)', 'Height (m HAE)', 'Height (m MSL)', 'Classification',
       'Signal Confidence', 'Solar Elevation (deg)', 'point_type'],
      dtype='object')


In [4]:
from utils.data_io import reproject2


s2a_path = Path(
    "data\sentinel-2\subset_1_of_S2A_MSIL2A_20250106T031121_N0511_R075_T49QCD_20250106T061847_s2resampled.tif"
)

s2a_wgs84_path = Path(
    "data\sentinel-2\subset_1_of_S2A_MSIL2A_20250106T031121_N0511_R075_T49QCD_20250106T061847_s2resampled_wgs84.tif"
)

# reproject the raster to WGS84
try:
    reproject2(s2a_path, s2a_wgs84_path, epsg=4326)
except FileExistsError:
    print(f"{s2a_wgs84_path} already exists. Skipping reprojection.")


with rasterio.open(s2a_wgs84_path, mode="r") as src:
    raster_data = get_value_from_raster(
        src, ds["Longitude (deg)"].values, ds["Latitude (deg)"].values, index=0
    )


print(raster_data.shape)

data\sentinel-2\subset_1_of_S2A_MSIL2A_20250106T031121_N0511_R075_T49QCD_20250106T061847_s2resampled_wgs84.tif already exists. Skipping reprojection.
EPSG:4326
(12, 67)


In [5]:
dem_path = Path("data/dem/GMRT_resample.tif")

real_height = []
with rasterio.open(dem_path, mode="r") as src:
    values = get_value_from_raster(
        src, ds["Longitude (deg)"].values, ds["Latitude (deg)"].values, index=1
    )
    real_height.extend(values)



print("lon:" + str(ds["Longitude (deg)"].values[0]) + "\nlat:" + str(ds["Latitude (deg)"].values[0]))
print(real_height[0:3])

EPSG:4326
lon:109.23262797807338
lat:21.399754751983647
[-1.0057216, -1.0057893, -1.0063345]


# 拼接数据集

In [9]:
from utils.data_io import save_csv


add_data = {}

for i in range(raster_data.shape[0]):
    add_data["B" + str(i + 1)] = raster_data[i, :]

add_data["real_height"] = real_height

print(f"add_data index:{list(add_data.keys())}")

ml_data = ds.assign(**add_data)
print(ml_data.columns)

save_csv(ml_data, data_name="dataset", tag="train", backup=True)

add_data index:['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'real_height']
Index(['Latitude (deg)', 'Longitude (deg)', 'UTM Easting (m)',
       'UTM Northing (m)', 'Height (m MSL)', 'B1', 'B2', 'B3', 'B4', 'B5',
       'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'real_height'],
      dtype='object')
dataset_train.csv already exists, will be overwritten!


False